# ASA Hugging Face Demo

This notebook downloads the published checkpoint from Hugging Face and runs a CPU-only demo.


In [ ]:
# Environment & setup
import os
import sys
import subprocess

repo_dir = 'ASA'
if not os.path.isdir(repo_dir):
    subprocess.run(['git', 'clone', 'https://github.com/digitaldaimyo/ASA.git'], check=True)
os.chdir(repo_dir)
subprocess.run([sys.executable, '-m', 'pip', 'install', '-e', '.'], check=True)


In [ ]:
# Run demo
import torch
from asa.load_pretrained import load_pretrained

model, report, cfg = load_pretrained(
    repo_id='DigitalShogun/ASA-ASM-wikitext103-raw',
    filename=None,
    variant='baseline',
    device='cpu',
)
print('Checkpoint source:', report['state_dict_source'])
print('Allowlisted gaps:', {
    'missing': len(report['allowed_missing']),
    'unexpected': len(report['allowed_unexpected']),
    'mismatched': len(report['allowed_mismatched']),
})

seq_len = 32
input_ids = torch.randint(0, cfg.vocab_size, (1, seq_len))
with torch.no_grad():
    logits, _ = model(input_ids)
print('Logits shape:', tuple(logits.shape))
assert logits.shape == (1, seq_len, cfg.vocab_size)
assert torch.isfinite(logits).all()
